In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # normalize image to [-1, 1]

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# 10 classes in total
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck') 

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# load test data (note that the data has been transformed already)
test_images = torch.load('test_image.pt')

In [10]:
# specify the GPU device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
import torchvision.models as models

In [12]:
#to try ResNet models with different number of layers
def create_resnet(num_layers, dropout_rate):
    if num_layers == 18:
        net = models.resnet18(pretrained=False,num_classes=10).to(device)
    elif num_layers == 34:
        net = models.resnet34(pretrained=False,num_classes=10).to(device)
    elif num_layers == 50:
        net = models.resnet50(pretrained=False,num_classes=10).to(device)
    else:
        raise ValueError("Unsupported number of layers")

    num_features = net.fc.in_features
    net.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_features, 10)
    )
    return net

In [24]:
#this is the final model for the kaggle submission
#num_layers = 18, dropout_rate = 0.29578761633364997
#net = models.resnet18(pretrained=False, num_classes=10).to(device)
net = create_resnet(num_layers=18, dropout_rate=0.29578761633364997).to(device)

In [13]:
import torch.optim as optim
import torch.nn as nn
import optuna

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0007836827942082852, momentum=0.9882941058204819, weight_decay=9.116296060594481e-05)

#results from two attemps with hyperparameter tuning using optuna
#Best trial: score 66.76, params {'lr': 0.0008323950770691731, 'momentum': 0.9659751984631002, 'weight_decay': 0.0001880406194490467}  layers = 18, drop out rate = 0.1
#Best trial: score 69.5, params {'lr': 0.0007836827942082852, 'momentum': 0.9882941058204819, 'weight_decay': 9.116296060594481e-05, 'num_layers': 18, 'dropout_rate': 0.29578761633364997}

best_acc = 0.0  # track the best accuracy

# Training loop
for epoch in range(20): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} loss: {running_loss / len(trainloader)}")
    
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1} Accuracy: {acc}%")

    # Check if this model has the best accuracy
    if acc > best_acc:
        best_acc = acc
        torch.save(net.state_dict(), 'best_model.pth')  # save the best model
        
print('Finished Training')

In [28]:
import numpy as np

In [ ]:
#make predictions to the kaggle test dataset
#model = models.resnet18(pretrained=False, num_classes=10).to(device)
model = create_resnet(num_layers=18, dropout_rate=0.29578761633364997).to(device)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set the model to evaluation mode

# Use the model for predictions
with torch.no_grad():
    test_images = test_images.to(device)
    output = model(test_images)
    _, predicted = torch.max(output.data, 1)
    predicted = np.array([classes[i] for i in predicted])
    
print(predicted)   
    
submission = pd.DataFrame()
submission['label'] = predicted
submission.to_csv("submission1026.csv", index=True, index_label='id')
submission

In [14]:
###########with hyperparameter tuning#################
import optuna
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [ ]:
 def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    momentum = trial.suggest_float('momentum', 0.5, 0.99)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-4, log=True)
    num_layers = trial.suggest_categorical('num_layers', [18, 34, 50])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    net = create_resnet(num_layers, dropout_rate).to(device) 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    best_acc = 0.0

    for epoch in range(15):  #after several attempts, it seems like the loss become stable after 10 epochs
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1} Accuracy: {acc}%")
        if acc > best_acc:
            best_acc = acc

    return best_acc  

# optimization
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=50)  #for the kaggle project, the tuning was based on 50 trials.

# Print best results
print(f'Best trial: score {study.best_value}, params {study.best_params}')